In [ ]:
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import json
import pickle
import numpy as np
from django.contrib.auth.models import User as DjangoUser
from backend.models import UserPreferences, Video, VideoRating
from backend.rating_fields import VIDEO_FIELDS
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
results_dir = '/home/ts/ray_results/featureless_tournesol_lambda'

In [ ]:
exps = list(filter(lambda x: x.startswith('experiment_'), sorted(os.listdir(results_dir))))

In [ ]:
result = {}

for exp in tqdm(exps):
    params_path = os.path.join(results_dir, exp, 'params.json')
    
    if not os.path.isfile(params_path):
        continue
    
    with open(params_path, 'r') as f:
        params = json.loads(f.read())

    ckpt_path = os.path.join(results_dir, exp, 'checkpoint_50000', 'learner_ckpt.pkl')

    if not os.path.isfile(ckpt_path):
        continue

    with open(ckpt_path, 'rb') as f:
        ckpt = pickle.load(f)

    all_ratings = ckpt['aggregator']['ratings']
    idx = all_ratings['layer']['idx']
    data = all_ratings['data']
    objects = all_ratings['objects']
    features = all_ratings['features']
    experts = all_ratings['experts']

    common_expert = experts[-1]
    common_expert_id = len(experts) - 1

    def get_agg_video_scores(video_id):
        video_id = objects.index(video_id)
        video_scores = []
        for feature_id, feature in enumerate(features):
            idx_v = idx.get_key((common_expert_id, video_id, feature_id))
            score_v = data[idx_v]
            video_scores.append(score_v)
        return np.array(video_scores, dtype=np.float32)

    scores = np.array([get_agg_video_scores(vid) for vid in objects])
    
    result[exp] = {'params': params, 'scores': scores, 'objects': objects, 'features': features, 'experts': experts}

In [ ]:
all_objects = [x['objects'] for x in result.values()]
assert all([all_objects[0] == o for o in all_objects])
all_features = [x['features'] for x in result.values()]
assert all([all_features[0] == o for o in all_features])
#all_experts = [x['experts'] for x in result.values()]
#assert all([all_experts[0] == o for o in all_experts])

In [ ]:
names = [Video.objects.get(video_id=vid).name for vid in df.index]

In [ ]:
for f in tqdm(VIDEO_FIELDS):
# f = VIDEO_FIELDS[0]
    mus = set()
    lambdas = set()

    df_by_param = {}

    for res in result.values():
        params = res['params']
        scores = res['scores']
        all_params.append(params)

        lam = params['_gin__lam__grid_search']
        mu = params['_gin__mu__grid_search']

        param_desc = f"lam={lam} mu={mu}"

        mus.add(mu)
        lambdas.add(lam)

        df = pd.DataFrame(scores, columns=all_features[0], index=[x[:5] for x in all_objects[0]])
        df['name'] = [x[:10] for x in names]
        #df.index = [x for x in df.index]

        df_top = df.sort_values(f, ascending=False)[:5]
        df_bot = df.sort_values(f, ascending=False)[-5:]
        top_bot = pd.concat([df_top, df_bot])
        df_top_bot = top_bot[['name', f]]
        df_by_param[(lam, mu)] = df_top_bot

    html = f'<h1>{f}</h1><table>'

    html += '<tr><th></th>'
    for mu in sorted(mus):
        html += f'<th>mu={mu}</th>'
    html += '</tr>'

    for lam in sorted(lambdas):
        html += "<tr>"

        html += f"<th>lam={lam}</th>"

        for mu in sorted(mus):
            key = (lam, mu)
            if key not in df_by_param:
                html += "<td>--</td>"
            else:
                html += f"<td>{df_by_param[key].to_html()}</td>"

        html += "</tr>"

    html += '</table>'

    with open(f'results_{f}.html', 'w') as f:
        f.write(html)

In [ ]:
from IPython.display import HTML, display

In [ ]:
html

In [ ]:
display(f"<p>sdf</p>")

In [ ]:
param_desc

In [ ]:
df_top_bot

In [ ]:
features, experts[:10], objects[:10]